In [ ]:
create or replace table driver_table
as
WITH median_time AS (
    SELECT 
        PERCENTILE_CONT(0.5) WITHIN GROUP (
            ORDER BY CASE 
                WHEN POSITION(':' IN TIME) > 0 THEN 
                    SPLIT_PART(TIME, ':', 1)::INT * 60 + SPLIT_PART(TIME, ':', 2)::FLOAT
                ELSE 
                    TIME::FLOAT 
            END
        ) AS median_value
    FROM pitstops
    WHERE TIME IS NOT NULL
)
SELECT DISTINCT 
    a.CAR,
    C.DETAIL,
    a.DRIVER,
    B.DRIVERCODE,
    D.GAP,
    a.GRAND_PRIX,
    C.LAP,
    B.NATIONALITY,
    A.RACE_POSITION,
    C.STOPS,
    CASE 
        WHEN POSITION(':' IN C.TIME) > 0 THEN 
            SPLIT_PART(COALESCE(C.TIME, TO_CHAR(median_time.median_value)), ':', 1)::INT * 60 + SPLIT_PART(C.TIME, ':', 2)::FLOAT
        ELSE 
            COALESCE(C.TIME, TO_CHAR(median_time.median_value))::FLOAT 
    END AS TIME,
    A.YEAR
FROM DRIVER_DETAILS a
LEFT JOIN driver_standings b 
    ON a.driver = b.driver AND a.year = b.year AND a.car = b.car
LEFT JOIN pitstops c 
    ON a.driver = c.driver AND a.year = c.year AND a.car = c.car AND a.grand_prix = c.grand_prix
LEFT JOIN practices d 
    ON a.driver = d.driver AND a.car = d.car AND a.year = d.year AND a.grand_prix = d.grand_prix AND a.race_position = d.pos
LEFT JOIN sprint_results e 
    ON a.driver = e.driver AND a.year = e.year AND a.race_position = e.pos AND a.car = e.car AND a.grand_prix = e.grand_prix
LEFT JOIN race_details f
    ON a.driver = f.driver AND a.year = f.year AND a.race_position = f.pos AND a.car = f.car AND a.grand_prix = f.grand_prix
CROSS JOIN median_time; -- Use CROSS JOIN for scalar CTE results

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE temp_driver_table AS
WITH median_time AS (
    SELECT 
        MEDIAN(CASE 
            WHEN POSITION(':' IN TIME) > 0 THEN 
                SPLIT_PART(TIME, ':', 1)::INT * 60 + SPLIT_PART(TIME, ':', 2)::FLOAT
            ELSE 
                TIME::FLOAT 
        END) AS median_value
    FROM pitstops
    WHERE TIME IS NOT NULL
),
most_frequent_values AS (
    SELECT 
        (SELECT CAR FROM DRIVER_DETAILS GROUP BY CAR ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_car,
        (SELECT DETAIL FROM pitstops GROUP BY DETAIL ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_detail,
        (SELECT NATIONALITY FROM driver_standings GROUP BY NATIONALITY ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_nationality,
        (SELECT GAP FROM practices GROUP BY GAP ORDER BY COUNT(*) DESC LIMIT 1) AS most_frequent_gap,
        MEDIAN(LAP) AS median_lap,
        MEDIAN(STOPS) AS median_stops
    FROM pitstops
)
SELECT DISTINCT 
    COALESCE(a.CAR, most_frequent_values.most_frequent_car, 'unknown') AS CAR,
    COALESCE(C.DETAIL, most_frequent_values.most_frequent_detail, 'unknown') AS DETAIL,
    COALESCE(a.DRIVER, 'unknown') AS DRIVER,
    COALESCE(B.DRIVERCODE, 'unknown') AS DRIVERCODE,
    COALESCE(D.GAP, most_frequent_values.most_frequent_gap, 'unknown') AS GAP,
    COALESCE(a.GRAND_PRIX, 'unknown') AS GRAND_PRIX,
    COALESCE(C.LAP, most_frequent_values.median_lap, 0) AS LAP,
    COALESCE(B.NATIONALITY, most_frequent_values.most_frequent_nationality, 'unknown') AS NATIONALITY,
    -- Replace NULL RACE_POSITION with 'unknown'
    COALESCE(CAST(A.RACE_POSITION AS VARCHAR), 'unknown') AS RACE_POSITION,
    COALESCE(C.STOPS, most_frequent_values.median_stops, 0) AS STOPS,
    CASE 
        WHEN POSITION(':' IN C.TIME) > 0 THEN 
            SPLIT_PART(COALESCE(C.TIME, TO_CHAR(median_time.median_value)), ':', 1)::INT * 60 + SPLIT_PART(C.TIME, ':', 2)::FLOAT
        ELSE 
            COALESCE(C.TIME, TO_CHAR(median_time.median_value), 'unknown')::FLOAT 
    END AS TIME,
    COALESCE(A.YEAR, 'unknown') AS YEAR
FROM DRIVER_DETAILS a
LEFT JOIN driver_standings b 
    ON a.driver = b.driver AND a.year = b.year AND a.car = b.car
LEFT JOIN pitstops c 
    ON a.driver = c.driver AND a.year = c.year AND a.car = c.car AND a.grand_prix = c.grand_prix
LEFT JOIN practices d 
    ON a.driver = d.driver AND a.car = d.car AND a.year = d.year AND a.grand_prix = d.grand_prix AND a.race_position = d.pos
LEFT JOIN sprint_results e 
    ON a.driver = e.driver AND a.year = e.year AND a.race_position = e.pos AND a.car = e.car AND a.grand_prix = e.grand_prix,
most_frequent_values, median_time;